In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly

     |████████████████████████████████| 122kB 1.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests

from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         376 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-cert

# Data adquisition

The coordinates of the neighborhoods are extracted from the [Seville city hall webpage](https://www.sevilla.org/servicios/servicio-de-estadistica/callejero/datos-alfanumericos), and transformed in the file seville_hoods.csv

In [3]:
neighborhoods = pd.read_csv("seville_hoods.csv")
neighborhoods.tail()

,Unnamed: 0,Name,Latitude,Longitude
103,103,Aeropuerto Viejo,37.415896,-5.946192
104,104,"Palacio Congresos, Urbadiez, Entrepuentes, Jar...",37.396032,-5.936959
105,105,"Colores, Entreparques",37.392615,-5.873559
106,106,Valdezorras,37.427943,-5.914069
107,107,Palmete,37.377730,-5.931119


The coordinates of the city can be obtained with geopy 

In [4]:
address = 'Seville'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

The map of the city and the location of the neighborhoods are plotted

In [5]:
map_seville = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Name']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seville)  
    
map_seville

The credentials of Foursquare are stored

In [9]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ******
CLIENT_SECRET:******


A function to get the venues of each place is created and used

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
seville_venues = getNearbyVenues(names=neighborhoods['Name'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Bellavista
El Plantinar
El Cerro
Retiro Obrero
La Oliva
Bda. Pino Montano
Torreblanca
Isla de Garza
Tablada
Las Almenas
Triana Oeste
Los Remedios
El Cano, Los Bermejales
El Porvenir
Pedro Salvador, Las Palmeritas
Nervión
San Bartolomé
Doctor Barraquer, G. Renfe, Policlínico
Los Pájaros
San Pablo C
El Gordillo
El Tardón, El Carmen
Sector Sur, La Palmera, Reina Mercedes
Bami
Tiro de Línea, Santa Genoveva
Felipe II, Los Diez Mandamientos
Rochelambert
Feria
Macarena 3 Huertas, Macarena 5
El Rocío
Los Príncipes, La Fontanilla
Las Huertas
Santa María de Ordas, San Nicolás
Arbol Gordo
Santa Aurelia, Cantábrico, Atlántico, La Romería
Santa Cruz
Giralda Sur
La Plata
Triana Casco Antiguo
Museo
La Florida
San Lorenzo
La Paz, Las Golondrinas
Hermandades, La Carrasca
Polígono Norte
La Barzola
Los Arcos
Amate
Juan XXIII
Triana Este
San Vicente
Encarnación, Regina
Barrio León
San Roque
San Julián
Carretera de Carmona, María Auxiliadora, Fontanal
San José Obrero
San Gil
Cisneo Alto, Santa María de Gra

The data are verified

In [10]:
print(seville_venues.shape)
seville_venues.head()

(908, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellavista,37.335018,-5.97233,Burger King,37.335343,-5.969593,Fast Food Restaurant
1,Bellavista,37.335018,-5.97233,Harley Davidson Sevilla,37.334349,-5.969614,Motorcycle Shop
2,Bellavista,37.335018,-5.97233,Pa Comé,37.334255,-5.969604,Café
3,Bellavista,37.335018,-5.97233,Super Bazar,37.334168,-5.969459,Miscellaneous Shop
4,Bellavista,37.335018,-5.97233,Seven Fitness,37.333658,-5.969776,Gym


The venues are grouped by neighborhood

In [11]:
seville_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
seville_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bellavista,37.335018,-5.97233,Burger King,37.335343,-5.969593,Fast Food Restaurant
1,Bellavista,37.335018,-5.97233,Harley Davidson Sevilla,37.334349,-5.969614,Motorcycle Shop
2,Bellavista,37.335018,-5.97233,Pa Comé,37.334255,-5.969604,Café
3,Bellavista,37.335018,-5.97233,Super Bazar,37.334168,-5.969459,Miscellaneous Shop
4,Bellavista,37.335018,-5.97233,Seven Fitness,37.333658,-5.969776,Gym


In [12]:
print('There are {} uniques categories.'.format(len(seville_venues['Venue Category'].unique())))

There are 136 uniques categories.


Some places don't have enough venues to be representatives. A list with the neighborhoods with more than 5 results is made

In [13]:
seville_filtered = seville_venues[['Neighborhood', 'Venue']].groupby('Neighborhood').count()
seville_filtered = seville_filtered[seville_filtered['Venue']>4]
seville_filtered.head()

,Venue
Neighborhood,
Alfalfa,62
Arbol Gordo,8
Arenal,25
Bami,6
Barrio León,7


In [14]:
seville_filtered.shape

(55, 1)

A data frame is created replacing the features with dummy variables. Then it will be  by neighborhoods and filtered (number of venues > 4)

In [15]:
# one hot encoding
seville_onehot = pd.get_dummies(seville_venues[['Venue Category']], prefix="", prefix_sep="")
columns = list(seville_onehot.columns)
columns.remove('Neighborhood')

# add neighborhood column back to dataframe
seville_onehot['Neighborhood'] = seville_venues['Neighborhood'] 
new_columns = ['Neighborhood'] + columns
seville_onehot = seville_onehot[new_columns]

seville_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Athletics & Sports,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,Gift Shop,Golf Course,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Karaoke Bar,Lake,Latin American Restaurant,Lebanese Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Nightclub,Other Repair Shop,Palace,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Pizza Place,Plaza,Pool,Pub,Record Shop,Restaurant,Road,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bellavista,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
seville_grouped = seville_onehot.groupby('Neighborhood').mean().reset_index()
seville_grouped = seville_grouped[seville_grouped['Neighborhood'].isin(list(seville_filtered.index))]
seville_onehot.shape

(908, 136)

A function to get the most common venues of each neighborhood is defined

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seville_grouped['Neighborhood']

for ind in np.arange(seville_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seville_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alfalfa,Plaza,Tapas Restaurant,Clothing Store,Ice Cream Shop,Restaurant,Café,Spanish Restaurant,Coffee Shop,Hotel,Bookstore
1,Arbol Gordo,Pizza Place,Spanish Restaurant,Convenience Store,Athletics & Sports,Tapas Restaurant,Diner,Café,Chinese Restaurant,Fish Market,Fish & Chips Shop
2,Arenal,Tapas Restaurant,Ice Cream Shop,Spanish Restaurant,Restaurant,Wine Bar,Hotel,Performing Arts Venue,Mediterranean Restaurant,Café,Plaza
4,Bami,Spanish Restaurant,Soccer Field,Grocery Store,Tapas Restaurant,Plaza,Wine Bar,Event Space,Fish Market,Fish & Chips Shop,Fast Food Restaurant
5,Barrio León,Plaza,Dessert Shop,Spanish Restaurant,Camera Store,Pizza Place,Tapas Restaurant,Fish Market,Department Store,Deli / Bodega,Food


# Model

k-means clustering algorithm 

In [19]:
# set number of clusters
kclusters = 7
seville_grouped_clustering = seville_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seville_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([5, 5, 5, 0, 5, 2, 3, 5, 5, 5], dtype=int32)

In [20]:
# Insert the labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Merge the dataframe with the results to the original
seville_merged = neighborhoods[neighborhoods['Name'].isin(list(seville_filtered.index))]
seville_merged = seville_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

seville_merged.head()

,Unnamed: 0,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Bellavista,37.335018,-5.972330,3,Motorcycle Shop,Miscellaneous Shop,Gym,Fast Food Restaurant,Café,Wine Bar,Fish Market,Fish & Chips Shop,Farmers Market,Event Space
1,1,El Plantinar,37.374397,-5.972094,0,Spanish Restaurant,Brewery,Diner,Sporting Goods Shop,Fast Food Restaurant,Food & Drink Shop,Food,Fish Market,Fish & Chips Shop,Wine Bar
2,2,El Cerro,37.372246,-5.958409,0,Spanish Restaurant,Ice Cream Shop,Restaurant,Bar,Wine Bar,Farmers Market,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant
3,3,Retiro Obrero,37.402457,-5.979158,2,Coffee Shop,Tapas Restaurant,Lounge,Ice Cream Shop,Spanish Restaurant,Wine Bar,Farmers Market,Food,Fish Market,Fish & Chips Shop
11,11,Los Remedios,37.374236,-5.999090,5,Mexican Restaurant,Fast Food Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Gastropub,Event Space,Fish Market,Fish & Chips Shop,Farmers Market


In [21]:
seville_merged.dropna(axis=0, inplace=True)
seville_merged["Cluster Labels"].astype(int)
seville_merged.shape

(55, 15)

Map with the classified neighborhoods

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seville_merged['Latitude'], seville_merged['Longitude'], seville_merged['Name'], seville_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The different clusters

In [30]:
seville_merged.loc[seville_merged['Cluster Labels'] == 0, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,El Plantinar,Spanish Restaurant,Brewery,Diner,Sporting Goods Shop,Fast Food Restaurant,Food & Drink Shop,Food,Fish Market,Fish & Chips Shop,Wine Bar
2,El Cerro,Spanish Restaurant,Ice Cream Shop,Restaurant,Bar,Wine Bar,Farmers Market,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant
23,Bami,Spanish Restaurant,Soccer Field,Grocery Store,Tapas Restaurant,Plaza,Wine Bar,Event Space,Fish Market,Fish & Chips Shop,Fast Food Restaurant
73,Huerta de Santa Teresa,Spanish Restaurant,Hotel,Restaurant,Mediterranean Restaurant,Tapas Restaurant,Brewery,Fast Food Restaurant,Food & Drink Shop,Food,Fish Market
87,"San Carlos, Tartessos",Spanish Restaurant,Brewery,Café,Tapas Restaurant,Food & Drink Shop,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [31]:
seville_merged.loc[seville_merged['Cluster Labels'] == 1, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,El Rocío,Diner,Fried Chicken Joint,Dessert Shop,Pizza Place,Tapas Restaurant,Bakery,Restaurant,Farmers Market,Fish Market,Fish & Chips Shop
43,"Hermandades, La Carrasca",Diner,Fried Chicken Joint,Pizza Place,Tapas Restaurant,Bakery,Restaurant,Event Space,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [32]:
seville_merged.loc[seville_merged['Cluster Labels'] == 2, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Retiro Obrero,Coffee Shop,Tapas Restaurant,Lounge,Ice Cream Shop,Spanish Restaurant,Wine Bar,Farmers Market,Food,Fish Market,Fish & Chips Shop
13,El Porvenir,Tapas Restaurant,Spanish Restaurant,Restaurant,Gym / Fitness Center,Grocery Store,Mexican Restaurant,Tennis Court,Ice Cream Shop,Bar,Café
16,San Bartolomé,Tapas Restaurant,Museum,Hotel,Performing Arts Venue,Bakery,Spa,Wine Bar,Event Space,Fish Market,Fish & Chips Shop
17,"Doctor Barraquer, G. Renfe, Policlínico",Tapas Restaurant,Supermarket,Sandwich Place,Beer Garden,Pub,Spanish Restaurant,Event Space,Fish Market,Fish & Chips Shop,Fast Food Restaurant
24,"Tiro de Línea, Santa Genoveva",Café,Gym / Fitness Center,Tapas Restaurant,Gastropub,Farmers Market,Wine Bar,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant
28,"Macarena 3 Huertas, Macarena 5",Tapas Restaurant,Coffee Shop,Fast Food Restaurant,Spanish Restaurant,Café,Wine Bar,Event Space,Food,Fish Market,Fish & Chips Shop
38,Triana Casco Antiguo,Tapas Restaurant,Spanish Restaurant,Restaurant,Bar,Brewery,Deli / Bodega,Lounge,Market,Fish & Chips Shop,Snack Place
41,San Lorenzo,Tapas Restaurant,Gay Bar,Spanish Restaurant,Gym / Fitness Center,Mexican Restaurant,Hotel,Café,Indian Restaurant,Bar,Nightclub
49,Triana Este,Tapas Restaurant,Seafood Restaurant,Gastropub,Chinese Restaurant,Spanish Restaurant,Pub,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
60,"León XIII, Los Naranjos",Tapas Restaurant,Coffee Shop,Spanish Restaurant,Café,Wine Bar,Event Space,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [33]:
seville_merged.loc[seville_merged['Cluster Labels'] == 3, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bellavista,Motorcycle Shop,Miscellaneous Shop,Gym,Fast Food Restaurant,Café,Wine Bar,Fish Market,Fish & Chips Shop,Farmers Market,Event Space


In [34]:
seville_merged.loc[seville_merged['Cluster Labels'] == 4, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Huerta de la Salud,Hotel,Snack Place,Train Station,Gym Pool,Event Space,Food & Drink Shop,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [35]:
seville_merged.loc[seville_merged['Cluster Labels'] == 5, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Los Remedios,Mexican Restaurant,Fast Food Restaurant,Ice Cream Shop,Pizza Place,Restaurant,Gastropub,Event Space,Fish Market,Fish & Chips Shop,Farmers Market
12,"El Cano, Los Bermejales",Tapas Restaurant,Restaurant,Pizza Place,Spanish Restaurant,Bar,Café,Lounge,Snack Place,Italian Restaurant,Beer Garden
15,Nervión,Grocery Store,Café,Coffee Shop,Bar,Fast Food Restaurant,Pub,Plaza,Mediterranean Restaurant,Tapas Restaurant,Diner
22,"Sector Sur, La Palmera, Reina Mercedes",College Quad,Bar,Sandwich Place,Brewery,Breakfast Spot,Café,Fast Food Restaurant,Pizza Place,Bakery,History Museum
27,Feria,Tapas Restaurant,Café,Mediterranean Restaurant,Pizza Place,Cocktail Bar,Bar,Ice Cream Shop,Plaza,French Restaurant,Japanese Restaurant
33,Arbol Gordo,Pizza Place,Spanish Restaurant,Convenience Store,Athletics & Sports,Tapas Restaurant,Diner,Café,Chinese Restaurant,Fish Market,Fish & Chips Shop
34,"Santa Aurelia, Cantábrico, Atlántico, La Romería",Park,Café,Mexican Restaurant,Spanish Restaurant,Plaza,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Event Space,Electronics Store
35,Santa Cruz,Spanish Restaurant,Plaza,Restaurant,Tapas Restaurant,Cafeteria,Ice Cream Shop,Italian Restaurant,Sandwich Place,Hostel,Mexican Restaurant
36,Giralda Sur,Spanish Restaurant,Park,Cocktail Bar,Bookstore,Pizza Place,Gastropub,Diner,Dessert Shop,Italian Restaurant,Tapas Restaurant
39,Museo,Hotel,Restaurant,Spanish Restaurant,Boarding House,Coffee Shop,Nightclub,Sandwich Place,Indie Movie Theater,Park,Ice Cream Shop


In [36]:
seville_merged.loc[seville_merged['Cluster Labels'] == 6, seville_merged.columns[[1] + list(range(5, seville_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Villegas,Breakfast Spot,Bus Stop,Tapas Restaurant,Bakery,Brewery,Fish & Chips Shop,French Restaurant,Football Stadium,Food & Drink Shop,Food
91,San Diego,Breakfast Spot,Tapas Restaurant,Beer Bar,Bar,Brewery,Electronics Store,Event Space,Diner,Farmers Market,Dessert Shop
